In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

In [3]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

In [4]:
# import matplotlib.pyplot as plt
import numpy as np
import pathlib

In [5]:
IMG_SIZE = (150, 150)

In [6]:
train_data_path = './data/seg_train/seg_train/'
test_data_path = './data/seg_test/seg_test/'
pred_data_path = './data/seg_pred/'


In [7]:
data_dir = pathlib.Path(train_data_path)
class_names = np.array([sorted(item.name for item in data_dir.glob('*'))])

In [8]:
class_names

array([['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']],
      dtype='<U9')

In [9]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    zoom_range=0.2,
    rotation_range=0.5
)

In [10]:
train_data = datagen.flow_from_directory(
    train_data_path,
    target_size=IMG_SIZE,
)
test_data = datagen.flow_from_directory(
    test_data_path,
    target_size=IMG_SIZE,
)
pred_data = datagen.flow_from_directory(
    pred_data_path,
    target_size=IMG_SIZE,
)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Found 7301 images belonging to 1 classes.


In [11]:
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir='./tb_model_1', histogram_freq=0, write_graph=True, write_images=True)

In [12]:
model_1 = tf.keras.Sequential([
    layers.Conv2D(32, 6, padding='same', activation='relu', input_shape=(150,150, 3)),
    layers.Conv2D(32, 6, padding='same', activation='relu'),
    layers.Conv2D(32, 6, padding='same', activation='relu'),
    layers.MaxPool2D(),
    layers.Conv2D(64, 5, padding='same', activation='relu'),
    layers.Conv2D(64, 5, padding='same', activation='relu'),
    layers.Conv2D(64, 5, padding='same', activation='relu'),
    layers.MaxPool2D(),
    layers.Conv2D(128, 4, padding='same', activation='relu'),
    layers.Conv2D(128, 4, padding='same', activation='relu'),
    layers.Conv2D(128, 4, padding='same', activation='relu'),
    layers.MaxPool2D(),
    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(6, activation='softmax'),
])

model_1.compile(
    optimizer=Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)



In [13]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      3488      
                                                                 
 conv2d_1 (Conv2D)           (None, 150, 150, 32)      36896     
                                                                 
 conv2d_2 (Conv2D)           (None, 150, 150, 32)      36896     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 32)       0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 75, 75, 64)        51264     
                                                                 
 conv2d_4 (Conv2D)           (None, 75, 75, 64)        102464    
                                                        

In [13]:
history_1 = model_1.fit(
    train_data,
    epochs=5,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps=len(test_data),
    callbacks=[tbCallBack]
)


Epoch 1/5
112/439 [======>.......................] - ETA: 25:28 - loss: 1.7949 - accuracy: 0.1702

KeyboardInterrupt: 

In [14]:
model_1.evaluate(pred_data)

KeyboardInterrupt: 

#### place for plotting history_1

In [19]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

# using functional api

In [16]:
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B3(include_top=False)

52606240/52606240 [==============================] - 60s 1us/step


In [18]:
# freezing weight to not change
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=(150, 150, 3), name='input_layer')

x = base_model(inputs)

x= tf.keras.layers.GlobalAveragePooling2D(name='global_avg_pooling')(x)

outputs = tf.keras.layers.Dense(6, activation='softmax', name='outputs')(x)

model_2 = tf.keras.Model(inputs, outputs)

In [ ]:
model_2.compile(
    optimizer=Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
history_2 = model_2.fit(
    train_data,
    epochs=5,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps=len(test_data),
    # callbacks=[tbCallBack]
)